In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import re

#### Import the data and get a high-level picture

In [4]:
df = pd.read_csv('sales.csv')
df.head()

,order_id,name,ordered_at,price,quantity,line_total
0,10000,"""ICE CREAM"" Peanut Fudge",2018-01-01 11:30:00,$3.50,3,$10.50
1,10000,"""ICE CREAM"" Peanut Fudge",2018-01-01 11:30:00,$3.50,1,$3.50
2,10001,"""SORBET"" Raspberry",2018-01-01 12:14:54,$2.50,2,$5.00
3,10001,NaN,2018-01-01 12:14:54,$1.50,1,$1.50
4,10001,"""CONE"" Dipped Waffle Cone",2018-01-01 12:14:54,$3.50,1,$3.50


In [5]:
df.shape

(29922, 6)

In [6]:
df.dtypes

order_id       int64
name          object
ordered_at    object
price         object
quantity       int64
line_total    object
dtype: object

#### TODO: Fix column datatypes

Change ordered_at to datetime

Change price and line_total to float

In [10]:
pd.to_datetime(df['ordered_at'])

0       2018-01-01 11:30:00
1       2018-01-01 11:30:00
2       2018-01-01 12:14:54
3       2018-01-01 12:14:54
4       2018-01-01 12:14:54
                ...        
29917   2018-06-26 03:56:13
29918   2018-03-03 10:06:21
29919   2018-04-05 17:33:24
29920   2018-07-20 09:10:01
29921   2018-07-13 09:20:21
Name: ordered_at, Length: 29922, dtype: datetime64[ns]

In [19]:
df['price'] = pd.to_numeric(df['price'],errors='coerce')
df['price']= df['price'].astype(float)
df['line_total'] = pd.to_numeric(df['line_total'],errors='coerce')
df['line_total']= df['line_total'].astype(float)

In [20]:
df.dtypes

order_id        int64
name           object
ordered_at     object
price         float64
quantity        int64
line_total    float64
dtype: object

#### TODO: drop if duplicated or null

In [21]:
df[df.duplicated()].shape[0]

584

In [ ]:
df.isnull().sum()

In [ ]:
df[df['name'].isnull()].head()

#### Sanity check for value ranges and to check assumptions

In [ ]:
df[(df['price'] * df['quantity']) != df['line_total']].shape[0]

In [ ]:
df[df['line_total'] < 0].shape[0]

#### TODO: 
Set line_total = price * quantity if different
Remove if line total < 0

In [ ]:
df.describe()

#### TODO: Get value between "" in name and put it in category column

In [ ]:
df.head()

#### Analysis, finally!

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
df.groupby('name')['line_total'].sum().sort_values(ascending=False).head(10).plot(kind='bar')
f.autofmt_xdate()
plt.show()